# Dimensionality reduction

## PCA by hand 

In [2]:
import pandas as pd

df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
    header=None, sep=',')

df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
df.dropna(how="all", inplace=True) # drops the empty line at file-end

df.tail()

# split data table into data X and class labels y

X = df.ix[:,0:4].values
y = df.ix[:,4].values

/home/amandae/anaconda3/envs/nibeta/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/amandae/anaconda3/envs/nibeta/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [6]:
# 1) Standardize data

from sklearn.preprocessing import StandardScaler
import numpy as np

X_std = StandardScaler().fit_transform(X)


In [9]:
# 2) covariance matrix
# dimension d x d (d = # of features)

cov_mat = np.cov(X_std.T) # mean_vec = np.mean(X_std, axis=0); cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)


Covariance matrix 
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]


In [10]:
# 3) eigendecomposition or SVD

# each column is an eigenvector

eig_vals, eig_vecs = np.linalg.eig(cov_mat)
u, s, v = np.linalg.svd(cov_mat)

print(eig_vals.shape)
print(eig_vecs.shape)
print(u.shape)
print(s.shape)
print(v.shape)

(4,)
(4, 4)
(4, 4)
(4,)
(4, 4)


All of these approaches yield the same eigenvectors and eigenvalue pairs:

* Eigendecomposition of the covariance matrix after standardizing the data.
* Eigendecomposition of the correlation matrix.
* Eigendecomposition of the correlation matrix after standardizing the data.



* Eigenvectors only define the *directions* of the new axis, since they all have the same unit length 1:

In [ ]:
for ev in range(len(eig_vecs)):
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(eig_vecs[:,ev]))
    # np.sum(np.square(ev))
print('Eigevectors all have unit length 1!')

In [ ]:
# 4) Sort pairs from highest to lowest

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort()
eig_pairs.reverse()

In [ ]:
# 5) Explained variance

tot  = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

plt.bar(['PC %s' %i for i in range(1,5)], var_exp)
plt.plot(['PC %s' %i for i in range(1,5)], cum_var_exp, color='r')
plt.scatter(['PC %s' %i for i in range(1,5)], cum_var_exp, color='r')
plt.title('Explained variance')
plt.ylabel('Explained variance (%)')
plt.show()

In [ ]:
# 6) Projection matrix 
# matrix of concatenated top k eigenvectors
# projection matrix = W = d x k

W = np.hstack((eig_pairs[0][1].reshape(4,1),
              eig_pairs[1][1].reshape(4,1)))
              
print(W)

In [ ]:
# 7) Transform data

Y = X_std.dot(W)  # Y = X * W


fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(1,1,1)
ax.scatter(Y[:, 0], Y[:, 1], c=iris.target, cmap=plt.cm.jet, edgecolor='k', s=50)
ax.set_title("First 2 PCA directions")
ax.set_xlabel("1st eigenvector")
ax.set_xticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.set_yticklabels([])

plt.show()

## SVD

https://blog.statsbot.co/singular-value-decomposition-tutorial-52c695315254

* A = U * S * V.T
* A = m * n 
* u = m * n orthogonal matrix
* s = n * n diagonal matrix (if n < m)
* v = n * n orthogonal matrix

In [ ]:
A = np.random.randn(4,3)
U, S, V = np.linalg.svd(A) # V here is V.T such that A = U*S*V
U = U[:,:-1] # where does the extra column come from?!?!?!?!?
ss = np.diag(S)
aa = np.dot(np.dot(U,ss),V)
np.testing.assert_array_almost_equal(A, aa)

In [ ]:
np.testing.assert_array_almost_equal?